# ETL Project

## Scrub down the olympic data set to limited scope. 

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import sqlite3 as sl

In [2]:
# Read in Olympic data

olympic_history = "./raw_olympic_data/athlete_events.csv"
o_history_df = pd.read_csv(olympic_history)
o_history_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [3]:
# Read in olympic region data

noc_regions = "./raw_olympic_data/noc_regions.csv"
regions_df = pd.read_csv(noc_regions)
regions_df.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [4]:
# drop notes column

reg_cols = ["NOC", "region"]
reg_cat_df = regions_df[reg_cols]
reg_cat_df.head()

,NOC,region
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra


In [5]:
# Check for null fields

reg_cat_df.count()

NOC       230
region    227
dtype: int64

In [6]:
# Drop all null fields from regions df

no_null_reg_cat_df = reg_cat_df.dropna(how='any')
no_null_reg_cat_df.head()

no_null_reg_cat_df.count()

NOC       227
region    227
dtype: int64

In [30]:

noc_df = no_null_reg_cat_df
print(len(noc_df))
noc_df.head()

227


,NOC,region
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra


In [18]:
# select columns for olympic events df

cols = ["Team","Games", "Season", "City", "Medal", "Sport", "Event", 'NOC']
olympic_cat_df = o_history_df[cols]
olympic_cat_df.head()

,Team,Games,Season,City,Medal,Sport,Event,NOC
0,China,1992 Summer,Summer,Barcelona,NaN,Basketball,Basketball Men's Basketball,CHN
1,China,2012 Summer,Summer,London,NaN,Judo,Judo Men's Extra-Lightweight,CHN
2,Denmark,1920 Summer,Summer,Antwerpen,NaN,Football,Football Men's Football,DEN
3,Denmark/Sweden,1900 Summer,Summer,Paris,Gold,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,DEN
4,Netherlands,1988 Winter,Winter,Calgary,NaN,Speed Skating,Speed Skating Women's 500 metres,NED


In [19]:
olympic_cat_df.count()

Team      271116
Games     271116
Season    271116
City      271116
Medal      39783
Sport     271116
Event     271116
NOC       271116
dtype: int64

In [20]:
# Drop all null fields from  olympic events df

no_null_olympic_cat_df = olympic_cat_df.dropna(how='any')
no_null_olympic_cat_df.head()

,Team,Games,Season,City,Medal,Sport,Event,NOC
3,Denmark/Sweden,1900 Summer,Summer,Paris,Gold,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,DEN
37,Finland,1920 Summer,Summer,Antwerpen,Bronze,Swimming,Swimming Men's 200 metres Breaststroke,FIN
38,Finland,1920 Summer,Summer,Antwerpen,Bronze,Swimming,Swimming Men's 400 metres Breaststroke,FIN
40,Finland,2014 Winter,Winter,Sochi,Bronze,Ice Hockey,Ice Hockey Men's Ice Hockey,FIN
41,Finland,1948 Summer,Summer,London,Bronze,Gymnastics,Gymnastics Men's Individual All-Around,FIN


In [21]:
# summary of medal counts

no_null_olympic_cat_df["Medal"].value_counts()

Gold      13372
Bronze    13295
Silver    13116
Name: Medal, dtype: int64

In [22]:
# Define final table for olympic data

new_data_df = no_null_olympic_cat_df
new_data_df.head()

,Team,Games,Season,City,Medal,Sport,Event,NOC
3,Denmark/Sweden,1900 Summer,Summer,Paris,Gold,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,DEN
37,Finland,1920 Summer,Summer,Antwerpen,Bronze,Swimming,Swimming Men's 200 metres Breaststroke,FIN
38,Finland,1920 Summer,Summer,Antwerpen,Bronze,Swimming,Swimming Men's 400 metres Breaststroke,FIN
40,Finland,2014 Winter,Winter,Sochi,Bronze,Ice Hockey,Ice Hockey Men's Ice Hockey,FIN
41,Finland,1948 Summer,Summer,London,Bronze,Gymnastics,Gymnastics Men's Individual All-Around,FIN


In [23]:
# List all sports in olympic event df

new_data_df["Sport"].unique()

array(['Tug-Of-War', 'Swimming', 'Ice Hockey', 'Gymnastics',
       'Alpine Skiing', 'Handball', 'Hockey', 'Rowing', 'Football',
       'Speed Skating', 'Sailing', 'Cycling', 'Fencing', 'Taekwondo',
       'Athletics', 'Canoeing', 'Water Polo', 'Wrestling',
       'Modern Pentathlon', 'Figure Skating', 'Golf', 'Softball',
       'Boxing', 'Basketball', 'Nordic Combined', 'Diving', 'Baseball',
       'Volleyball', 'Cross Country Skiing', 'Bobsleigh', 'Curling',
       'Shooting', 'Judo', 'Equestrianism', 'Tennis', 'Rugby Sevens',
       'Rhythmic Gymnastics', 'Weightlifting', 'Badminton',
       'Beach Volleyball', 'Ski Jumping', 'Rugby',
       'Short Track Speed Skating', 'Biathlon', 'Lacrosse',
       'Synchronized Swimming', 'Archery', 'Freestyle Skiing',
       'Triathlon', 'Polo', 'Luge', 'Table Tennis', 'Snowboarding',
       'Cricket', 'Skeleton', 'Racquets', 'Military Ski Patrol',
       'Croquet', 'Art Competitions', 'Roque', 'Alpinism', 'Trampolining',
       'Basque Pelota',

In [24]:
# select sports of interest

sports = ["Swimming", "Speed Skating"]
new_data_df = new_data_df.loc[new_data_df["Sport"].isin(sports)]

In [29]:
# Clean column name formats

new_data_df.columns = ['team', 'games', 'seasons', 'city', 'medal', 'sport', 'event', 'NOC']
new_data_df.head()

,team,games,seasons,city,medal,sport,event,NOC
37,Finland,1920 Summer,Summer,Antwerpen,Bronze,Swimming,Swimming Men's 200 metres Breaststroke,FIN
38,Finland,1920 Summer,Summer,Antwerpen,Bronze,Swimming,Swimming Men's 400 metres Breaststroke,FIN
110,Norway,1952 Winter,Winter,Oslo,Bronze,Speed Skating,"Speed Skating Men's 1,500 metres",NOR
113,Norway,1960 Winter,Winter,Squaw Valley,Gold,Speed Skating,"Speed Skating Men's 1,500 metres",NOR
210,Hungary,1936 Summer,Summer,Berlin,Bronze,Swimming,Swimming Men's 4 x 200 metres Freestyle Relay,HUN


In [26]:
# write to sql lite table

conn = sl.connect('olympic_db.sqlite')

new_data_df.to_sql('tab', conn, if_exists='replace', index=False)

In [27]:
noc_df.to_sql('noc', conn, if_exists='replace', index=False)